In [1]:
import os
import sys
from pandas_datareader import data as pdr
import pandas as pd
import fix_yahoo_finance as yf
import importlib
sys.path.append("../MachineLearningStocks/")
from download_historical_prices import *
from parsing_keystats import *
from backtesting import *
from stock_prediction import *
sys.path.append("../awsQLDB/")
os.environ["AWS_PROFILE"] = "Insight"
from create_ledger import *
from connect_to_ledger import *
from create_table import *
from create_index import * 
from insert_document import *
from hashlib import sha256
import pickle
sys.path.append("./")

/Users/kenmiyachi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [13]:
sp500 = build_sp500_dataset()
sp500['Open'] = sp500['Open'].apply(lambda x: mess_up(x))
sp500['Close'] = sp500['Close'].apply(lambda x: mess_up(x))
sp500

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-08-01,79.774220,99.529999,98.239998,75.293116,70.220901,49321000
2003-08-04,79.753962,99.000000,97.000000,26.966079,70.220901,55214100
2003-08-05,42.185310,98.760002,96.339996,12.034661,68.731087,61415600
2003-08-06,65.561552,98.059998,96.419998,14.674768,69.130257,50096900
2003-08-07,40.698259,98.070000,96.760002,59.858133,69.857361,43427400
2003-08-08,66.098342,98.550003,97.760002,16.236010,70.056938,27357300
2003-08-11,3.700996,99.040001,97.839996,45.086690,70.320663,34631400
2003-08-12,96.315280,99.589996,98.419998,70.000733,70.962265,43285600
2003-08-13,64.444631,99.849998,98.529999,6.592290,70.598709,36152000


In [3]:
sp500_df, stock_df = preprocess_price_data()
parsed_df = parse_keystats(sp500_df, stock_df)

Parsing progress:: 100%|██████████| 560/560 [04:20<00:00,  2.15tickers/s]


In [4]:
backtest()
# check_yahoo()
# current_df = forward()
# current_df.to_csv("forward_sample.csv", index=False)

Classifier performance
Accuracy score:  0.78
Precision score:  0.72

 Stock prediction performance report 
Total Trades: 173
Average return for stock predictions:  38.4 %
Average market return in the same period:  9.3% 
Compared to the index, our strategy earns  29.1 percentage points more


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [5]:
predict_stocks()

13 stocks predicted to outperform the S&P500 by more than 10%:
TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD


'TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD'

In [2]:
def collectionData(inputData, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    collection_data = [
        {
            'GitHash': commit_hash,
            'ScrapeTime': datetime.now(),
            'InputData': inputData,
            'outputHash': outHash
        }
    ]
    return collection_data

def trainingData(inputHash, model):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    training_data = [
        {
            'GitHash': commit_hash,
            'TrainTime': datetime.now(),
            'InputHash': inputHash,
            'Model': model
        }
    ]
    return training_data

def predictionData(inputHash, model, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    prediction_data = [
        {
            'GitHash': commit_hash,
            'InputHash': inputHash,
            'Model': model, 
            'PredictionTime': datetime.now(),
            'OutputHash': outHash
        }
    ]
    return prediction_data

In [3]:
import random
def mess_up(x):
    return x*random.random()
    

def data_collection(source="Yahoo Finance"):
    sp500 = build_sp500_dataset()
    sp500['Open'] = sp500['Open'].apply(lambda x: mess_up(x))
    sp500['Close'] = sp500['Close'].apply(lambda x: mess_up(x))
    sp500.to_csv("sp500_index.csv")
    s = pickle.dumps(sp500)
    outputHash = sha256(s).hexdigest()
    print(outputHash)
    # sp500_df, stock_df = preprocess_price_data()
    # parse_keystats(sp500_df, stock_df)
    data = collectionData(source, outputHash)
    dynamic_insert("StockMarketPrediction", "Collection", data)


def training(modelName="RandomForest-2020"): 
    model = backtest()
    s = pickle.dumps(model)
    modelHash = sha256(s).hexdigest()
    data = trainingData(modelHash, modelName)
    dynamic_insert("StockMarketPrediction", "Training", data)

def predict(modelName="RandomForest-2020"):
    stats = pd.read_csv('./new_keystats.csv')
    s = pickle.dumps(stats)
    inputHash = sha256(s).hexdigest()
    prediction = predict_stocks()
    outputHash = sha256(str.encode(prediction)).hexdigest()
    data = predictionData(inputHash, modelName, outputHash)
    dynamic_insert("StockMarketPrediction", "Prediction", data)

In [4]:
data_collection()
training("Test Model 1")
predict("Test Model 1")

[*********************100%***********************]  1 of 1 downloaded
376d6317611e15609b2a296b3c3fb161025fefc0c992c4da4db6dce8857cbf16
INFO:insert_document:Inserting some documents in the Collection table...
INFO:insert_document:Documents inserted successfully!
Classifier performance
Accuracy score:  0.79
Precision score:  0.70

 Stock prediction performance report 
Total Trades: 178
Average return for stock predictions:  35.3 %
Average market return in the same period:  9.2% 
Compared to the index, our strategy earns  26.1 percentage points more
INFO:insert_document:Inserting some documents in the Training table...
INFO:insert_document:Documents inserted successfully!
13 stocks predicted to outperform the S&P500 by more than 10%:
TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD
INFO:insert_document:Inserting some documents in the Prediction table...
INFO:insert_document:Documents inserted successfully!


In [38]:
print(b'Nice')

b'Nice'


In [22]:
sp500.to_csv("tbd.csv")

In [6]:
ks = pd.read_csv('./new_keystats.csv')
ks['Price'] = ks['Price'].apply(lambda x: mess_up(x))
ks['SP500'] = ks['SP500'].apply(lambda x: mess_up(x))